<a href="https://colab.research.google.com/github/megamishaa/translator/blob/main/translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers[sentencepiece] sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.3 MB/s eta 0:00:00


In [2]:
from IPython.core.async_helpers import sys
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

In [3]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-hi"

In [4]:
raw_datasets = load_dataset("cfilt/iitb-english-hindi")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

dataset_infos.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/190M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/85.7k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/500k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [5]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [6]:
raw_datasets = raw_datasets.shuffle(seed=42)
raw_datasets["train"] = raw_datasets["train"].select(range(20000))
raw_datasets["validation"] = raw_datasets["validation"].select(range(500))

preprocessing

In [7]:
tokenizer= AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [8]:
tokenizer("Hello, this is a sentence!")

{'input_ids': [12110, 2, 90, 23, 19, 8800, 61, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [9]:
max_input_length = 64
max_target_length = 64
source_lang = "en"
target_lang = "hi"

def preprocess_function(examples):
  inputs = [ex[source_lang] for ex in examples["translation"]]
  targets = [ex[target_lang] for ex in examples["translation"]]
  model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True
  )
  labels = tokenizer(
        targets,
        max_length=max_target_length,
        truncation=True
    )

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [10]:
preprocess_function(raw_datasets["train"][:2])

{'input_ids': [[68, 4, 61720, 3, 0], [643, 393, 46257, 12473, 142, 1857, 4874, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[44, 4499, 549, 197, 428, 1185, 2273, 1185, 61881, 260, 314, 44, 2703, 428, 40, 0], [44, 1, 1185, 2136, 716, 260, 428, 130, 0]]}

In [11]:
print(tokenizer)
print(type(tokenizer))


MarianTokenizer(name_or_path='Helsinki-NLP/opus-mt-en-hi', vocab_size=61950, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	61949: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)
<class 'transformers.models.marian.tokenization_marian.MarianTokenizer'>


In [12]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched = True)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [13]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tf_model.h5:   0%|          | 0.00/306M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [14]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
#num_train_epochs = 1

In [15]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model, return_tensors="tf")

In [16]:
#gneration_data_collector = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=64)

In [17]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    batch_size = batch_size,
    shuffle=True,
    collate_fn = data_collator,
)

In [18]:
validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

In [19]:
generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

In [20]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [21]:
model.fit(train_dataset, validation_data=validation_dataset, epochs=1)

1250/1250 [==============================] - 364s 264ms/step - loss: 2.7699 - val_loss: 2.2916


In [30]:
model.save_pretrained("translator_model")
tokenizer.save_pretrained("translator_model")


/usr/local/lib/python3.12/dist-packages/transformers/configuration_utils.py:461: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]]}
  warnings.warn(


('translator_model/tokenizer_config.json',
 'translator_model/special_tokens_map.json',
 'translator_model/vocab.json',
 'translator_model/source.spm',
 'translator_model/target.spm',
 'translator_model/added_tokens.json')

testing

In [24]:
pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 31.0 MB/s eta 0:00:00


In [32]:
tokenizer = AutoTokenizer.from_pretrained("translator_model")
model = TFAutoModelForSeq2SeqLM.from_pretrained("translator_model")

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at translator_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [33]:
input_text = "Hey! Tell me about Blue"
tokenized = tokenizer([input_text], return_tensors = 'np')
out = model.generate(**tokenized, max_length = 64)
print(out)

tf.Tensor(
[[61949    44   347    61    44  1056  7609   549    44  1056   174   549
     44  1056   174   549    44   906    44   800  2686  1042    44  1056
    174   549    44  1056   174   549    44  1056   174   549    44  1056
    174   549    44  1056   174   549    44  1056   174   549    44  1056
    174   549    44  1056   174   549    44  1056   174   549    44  1056
    174   549    44     0]], shape=(1, 64), dtype=int32)


In [34]:
with tokenizer.as_target_tokenizer():
  print(tokenizer.decode(out[0], skip_special_tokens=True))

ऐ! मैं में में के लिए में में में में में में में में में में
